In [588]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas
import numpy as np
import time
import seaborn as sns

pd.set_option('display.max_columns',75)

In [589]:
df = pd.read_csv('../datasets/total_data.csv',index_col=0)

In [590]:
df = df.drop(columns=['City','State','Metro'])
df = df.dropna(subset=['sale_date'])
df = df.fillna(0)

In [652]:
df[df.sale_price==0]

,RegionID,neighborhood,CountyName,SizeRank,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,Unnam,addr,lng,lat,apartment_number,borough,building_class_category,commercial_units,gross_square_feet,land_square_feet,residential_units,sale_date,sale_price,total_units,year_built,zip_code,rent_estimate


In [592]:
def rename_cols(df):
    new_cols = []
    for c in df.columns[0:4]:
        new_cols.append(c)
    for c in df.columns[4:-14]:
        d = c.split(' ')[0][:-3]
        new_cols.append(d)
    for c in df.columns[-14:]:
        new_cols.append(c)
    df.columns = new_cols

In [593]:
rename_cols(df)

In [594]:
def rename_sale_date(df):
    sale_dates = list(df.sale_date)
    new_dates = []
    for date in sale_dates:
        new_dates.append(str(date)[:7])
    df.sale_date = new_dates

In [595]:
rename_sale_date(df)

In [596]:
df.columns

Index(['RegionID', 'neighborhood', 'CountyName', 'SizeRank', '2012-01',
       '2012-02', '2012-03', '2012-04', '2012-05', '2012-06', '2012-07',
       '2012-08', '2012-09', '2012-10', '2012-11', '2012-12', '2013-01',
       '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07',
       '2013-08', '2013-09', '2013-10', '2013-11', '2013-12', '2014-01',
       '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-07',
       '2014-08', '2014-09', '2014-10', '2014-11', '2014-12', '2015-01',
       '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07',
       '2015-08', '2015-09', '2015-10', '2015-11', '2015-12', '2016-01',
       '2016-02', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07',
       '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01',
       '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07',
       '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01',
       '2018-02', '2018-03', '2018-04', '2018-05', '

In [597]:
def get_rent_estimates(df):
    rent_estimates = []
    start=time.time()
    for index, row in df.iterrows():
        for col in df.columns:
            if row.sale_date==row[col]:  
                if row[row.sale_date] !=0:
                    rent_estimate = row.sale_price/row[row.sale_date]/12
                    rent_estimates.append(round(rent_estimate))
                else:
                    rent_estimates.append(0)
        if (index+1)%5000 == 0:
            print('Runtime: {} seconds. Row: {}'.format(round(time.time()-start,2),index+1))
    print('\nTotal runtime: {} seconds'.format(round(time.time()-start,2)))
    return rent_estimates

In [598]:
re = get_rent_estimates(df)

Runtime: 22.94 seconds. Row: 5000
Runtime: 42.83 seconds. Row: 10000
Runtime: 60.57 seconds. Row: 15000
Runtime: 77.86 seconds. Row: 20000
Runtime: 99.31 seconds. Row: 25000
Runtime: 119.95 seconds. Row: 30000
Runtime: 138.97 seconds. Row: 35000
Runtime: 157.1 seconds. Row: 40000
Runtime: 175.04 seconds. Row: 45000
Runtime: 196.59 seconds. Row: 50000
Runtime: 220.08 seconds. Row: 55000
Runtime: 238.69 seconds. Row: 60000
Runtime: 256.49 seconds. Row: 65000
Runtime: 275.24 seconds. Row: 70000

Total runtime: 286.94 seconds


In [599]:
df['rent_estimate'] = re

In [600]:
df.head()

,RegionID,neighborhood,CountyName,SizeRank,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,Unnam,addr,lng,lat,apartment_number,borough,building_class_category,commercial_units,gross_square_feet,land_square_feet,residential_units,sale_date,sale_price,total_units,year_built,zip_code,rent_estimate
0,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,11909.0,360 MANHATTAN AVENUE,-73.957293,40.804399,A,1.0,15 CONDOS - 2-10 UNIT RESIDENTIAL,0.0,0.0,0.0,1.0,2012-05,695000.0,1.0,0.0,10026.0,2664
1,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,11910.0,360 MANHATTAN AVENUE,-73.957293,40.804399,C,1.0,15 CONDOS - 2-10 UNIT RESIDENTIAL,0.0,0.0,0.0,1.0,2012-03,665000.0,1.0,0.0,10026.0,2575
2,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,11911.0,360 MANHATTAN AVENUE,-73.957293,40.804399,H,1.0,15 CONDOS - 2-10 UNIT RESIDENTIAL,0.0,0.0,0.0,1.0,2012-02,721000.0,1.0,0.0,10026.0,2796
3,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,11912.0,360 MANHATTAN AVENUE,-73.957293,40.804399,B,1.0,15 CONDOS - 2-10 UNIT RESIDENTIAL,0.0,0.0,0.0,1.0,2013-08,955000.0,1.0,0.0,10026.0,3319
4,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,24794.0,130 MANHATTAN AVENUE,-74.054322,40.744161,0,1.0,01 ONE FAMILY HOMES,0.0,3264.0,1190.0,1.0,2012-12,2100000.0,1.0,1900.0,10025.0,8155


In [601]:
df.rent_estimate.describe()

count    7.315700e+04
mean     8.534942e+03
std      6.026241e+04
min      0.000000e+00
25%      2.120000e+03
50%      3.387000e+03
75%      6.486000e+03
max      6.772565e+06
Name: rent_estimate, dtype: float64

In [604]:
df[df.commercial_units>0].describe()

,RegionID,SizeRank,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,...,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,Unnam,lng,lat,borough,commercial_units,gross_square_feet,land_square_feet,residential_units,sale_price,total_units,year_built,zip_code,rent_estimate
count,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.0000,2950.000000,...,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.000000,2950.0,2.950000e+03,2950.000000,2950.000000,2.950000e+03,2950.0,2950.000000,2950.000000,2.950000e+03
mean,256018.504068,270.083390,20.682319,20.665424,20.614007,20.537634,20.374807,20.257119,20.183454,20.107556,20.273729,20.395756,20.435268,20.689688,21.095481,21.304492,21.316780,21.180441,20.981447,20.934054,20.923827,21.150820,21.137108,21.499776,21.639831,21.580837,21.668393,21.620631,21.763132,22.390214,22.814064,22.828986,22.744498,22.642146,22.543539,22.3768,22.570617,...,23.169471,23.497369,23.501905,23.507498,23.668803,23.734803,23.763549,24.069214,24.181322,24.601224,24.886217,24.832949,24.809146,24.758814,26.106058,26.070173,26.310142,26.630729,26.602769,26.584895,26.736081,26.866169,27.132678,27.289851,45449.076610,-77.377833,39.810053,2.396610,1.0,2.035166e+04,10327.805763,0.054915,1.285765e+07,1.0,1876.381356,10847.413898,4.511330e+04
std,60437.884095,429.867891,4.765406,4.684733,4.787566,4.847025,4.702912,4.727614,4.696017,4.690873,4.870685,4.942952,4.898101,5.136783,5.535461,5.506419,5.633193,5.531121,5.285780,5.218026,5.523196,5.677448,5.539552,5.731090,5.471512,5.255827,4.916350,4.952885,4.940327,5.101213,5.285724,5.211849,5.258164,5.169158,4.913726,4.6539,4.658680,...,4.541557,4.538195,4.493169,4.415080,4.497474,4.539406,4.488710,4.466614,4.402769,4.219111,4.043527,3.927410,3.902937,3.931496,4.195163,4.215824,4.224622,4.396882,4.456931,4.583170,4.590462,4.633576,4.893209,4.966712,21960.309011,15.128110,4.917888,0.918143,0.0,9.015938e+04,33808.406517,1.296251,5.782993e+07,0.0,354.694112,649.614720,1.915358e+05
min,193182.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.430000,14.320000,13.970000,13.960000,13.520000,13.380000,13.220000,13.100000,12.980000,13.3900,13.340000,...,12.870000,13.370000,13.570000,13.660000,13.720000,13.620000,13.620000,13.740000,13.990000,14.330000,14.600000,14.620000,14.620000,14.660000,15.420000,15.600000,15.990000,16.130000,15.990000,15.600000,15.760000,15.740000,16.040000,16.400000,100.000000,-124.218764,-42.885005,1.000000,1.0,0.000000e+00,0.000000,0.000000,1.014000e+05,1.0,0.000000,0.000000,0.000000e+00
25%,194737.000000,24.000000,17.130000,17.720000,17.010000,16.980000,16.890000,17.130000,17.100000,17.110000,17.130000,17.300000,17.260000,17.470000,17.870000,17.920000,18.010000,18.110000,18.150000,17.960000,17.760000,17.770000,17.930000,18.030000,17.730000,17.620000,17.660000,17.860000,17.960

In [605]:
df[df.rent_estimate<10000].describe()

,RegionID,SizeRank,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,2014-11,...,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,Unnam,lng,lat,borough,commercial_units,gross_square_feet,land_square_feet,residential_units,sale_price,total_units,year_built,zip_code,rent_estimate
count,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.00000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,...,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,62627.000000,6.262700e+04,62627.0,62627.00000,62627.000000,62627.000000
mean,261590.194261,348.078688,21.535633,21.450910,21.425797,21.371612,21.234756,21.199476,21.082775,20.946621,21.176422,21.297087,21.307022,21.560628,22.053928,22.238558,22.311400,22.213428,22.011439,21.937452,22.077656,22.460065,22.376388,22.722604,22.776102,22.654329,22.761863,22.733681,22.826361,23.571779,24.022199,23.95181,23.823269,23.705213,23.457153,23.162539,23.343417,...,23.610078,23.962284,23.944166,24.007594,24.153362,24.195820,24.166956,24.423372,24.508068,25.104649,25.486633,25.513483,25.469176,25.331710,26.735101,26.619016,26.830844,27.028752,26.924526,26.777351,26.877090,26.969673,27.088325,27.190343,35539.631772,-76.048872,39.748102,2.004742,0.025117,426.536334,605.320565,0.944497,1.006075e+06,1.0,1573.87903,10614.776630,3575.391716
std,53590.359980,478.969978,4.978171,4.936192,4.992252,5.029133,4.817967,4.877320,4.793257,4.853711,5.054413,5.190817,5.103843,5.418745,5.930750,5.734864,5.967418,5.829084,5.513807,5.299245,5.644687,5.903962,5.793350,5.979341,5.515143,5.223056,4.818284,4.849251,4.733407,4.870600,4.948873,4.75505,4.718967,4.610326,4.328317,4.086451,4.097021,...,3.757214,3.767082,3.748397,3.663207,3.705701,3.725952,3.688617,3.656577,3.631049,3.433537,3.229954,3.129621,3.097357,3.147206,3.401712,3.452103,3.481984,3.667168,3.737498,3.879317,3.904079,3.934226,4.252323,4.378485,21285.960234,19.038739,6.101214,0.999997,0.156482,2465.554006,5523.846161,0.270875,6.620659e+05,0.0,786.15868,727.016939,2175.409922
min,20239.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.430000,14.320000,13.970000,13.960000,13.520000,13.38000,13.220000,13.100000,12.980000,13.390000,13.340000,...,12.870000,13.370000,13.570000,13.660000,13.720000,13.620000,13.620000,13.740000,13.990000,14.330000,14.600000,14.620000,14.620000,14.660000,15.420000,15.600000,15.990000,16.130000,15.990000,15.600000,15.760000,15.740000,16.040000,16.400000,0.000000,-149.847176,-42.885005,1.000000,0.000000,0.000000,0.000000,0.000000,1.001980e+05,1.0,0.00000,0.000000,0.000000
25%,199001.000000,24.000000,18.340000,18.210000,17.630000,17.720000,17.540000,17.310000,17.430000,17.590000,17.590000,17.990000,18.070000,18.240000,18.560000,18.540000,18.800000,18.560000,18.450000,18.490000,18.260000,18.4

In [606]:
df.head()

,RegionID,neighborhood,CountyName,SizeRank,2012-01,2012-02,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,Unnam,addr,lng,lat,apartment_number,borough,building_class_category,commercial_units,gross_square_feet,land_square_feet,residential_units,sale_date,sale_price,total_units,year_built,zip_code,rent_estimate
0,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,11909.0,360 MANHATTAN AVENUE,-73.957293,40.804399,A,1.0,15 CONDOS - 2-10 UNIT RESIDENTIAL,0.0,0.0,0.0,1.0,2012-05,695000.0,1.0,0.0,10026.0,2664
1,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,11910.0,360 MANHATTAN AVENUE,-73.957293,40.804399,C,1.0,15 CONDOS - 2-10 UNIT RESIDENTIAL,0.0,0.0,0.0,1.0,2012-03,665000.0,1.0,0.0,10026.0,2575
2,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,11911.0,360 MANHATTAN AVENUE,-73.957293,40.804399,H,1.0,15 CONDOS - 2-10 UNIT RESIDENTIAL,0.0,0.0,0.0,1.0,2012-02,721000.0,1.0,0.0,10026.0,2796
3,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,11912.0,360 MANHATTAN AVENUE,-73.957293,40.804399,B,1.0,15 CONDOS - 2-10 UNIT RESIDENTIAL,0.0,0.0,0.0,1.0,2013-08,955000.0,1.0,0.0,10026.0,3319
4,270958,upper west side,New York,3,21.85,21.49,21.52,21.61,21.74,21.82,21.66,21.19,21.39,21.54,21.75,21.46,21.78,22.52,22.82,23.13,22.73,22.69,23.24,23.98,23.75,23.7,24.15,23.9,23.74,23.82,24.08,25.4,25.92,26.22,25.96,25.39,25.13,...,25.69,25.55,25.34,25.38,25.25,26.43,26.98,26.93,26.88,26.57,28.3,27.99,28.0,27.56,27.05,26.27,26.16,26.06,25.64,25.58,24794.0,130 MANHATTAN AVENUE,-74.054322,40.744161,0,1.0,01 ONE FAMILY HOMES,0.0,3264.0,1190.0,1.0,2012-12,2100000.0,1.0,1900.0,10025.0,8155


In [607]:
len(df)

73157

In [648]:
df2 = df[(df.commercial_units<1) & (df.rent_estimate>1)]
df2 = df2.drop(columns=['Unnam','commercial_units','residential_units','total_units','apartment_number', 'borough',
       'building_class_category', 'gross_square_feet', 'land_square_feet','year_built','2012-01',
       '2012-02', '2012-03', '2012-04', '2012-05', '2012-06', '2012-07',
       '2012-08', '2012-09', '2012-10', '2012-11', '2012-12', '2013-01',
       '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07',
       '2013-08', '2013-09', '2013-10', '2013-11', '2013-12', '2014-01',
       '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-07',
       '2014-08', '2014-09', '2014-10', '2014-11', '2014-12', '2015-01',
       '2015-02', '2015-03', '2015-04', '2015-05', '2015-06', '2015-07',
       '2015-08', '2015-09', '2015-10', '2015-11', '2015-12', '2016-01',
       '2016-02', '2016-03', '2016-04', '2016-05', '2016-06', '2016-07',
       '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01',
       '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07',
       '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01',
       '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07','SizeRank','CountyName','lng','lat'])

In [649]:
df2.describe()

,RegionID,sale_price,zip_code,rent_estimate
count,70146.000000,7.014600e+04,70146.000000,7.014600e+04
mean,263028.979372,2.032094e+06,10536.063710,7.004056e+03
std,52177.208603,1.487649e+07,744.198675,4.675933e+04
min,20239.000000,1.001980e+05,0.000000,1.860000e+02
25%,270811.000000,5.680000e+05,10019.000000,2.105000e+03
50%,270885.000000,9.200000e+05,10069.000000,3.321000e+03
75%,270958.000000,1.756481e+06,11216.000000,6.210750e+03
max,403222.000000,2.200000e+09,11416.000000,6.772565e+06


In [650]:
df2.columns

Index(['RegionID', 'neighborhood', 'addr', 'sale_date', 'sale_price',
       'zip_code', 'rent_estimate'],
      dtype='object')

In [651]:
df2.to_csv('rent_estimate_df.csv')